In [1]:
from PyPDF2 import PdfReader
import re

reader = PdfReader("series.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()

with open("series.txt", "w", encoding= 'utf-8') as f:
    f.write(text)

In [40]:
# series to data
import re

data = []

with open('series.txt', 'r', encoding= 'utf-8') as f:

    for line in f:
        
        # Look for the gender
        if re.search(r'Fem.*', line):
            gender = 'Femenino'
        elif re.search(r'Masc.*', line):
            gender = 'Masculino'
        
        # Look for the prueba
        if (p := re.search(r'Prueba\s.+\.,\s*(\d{2,4}m\s)\s*(Libre|Espalda|Braza|Mariposa|Estilos).*', line)):
            prueba = p.group(1) + p.group(2)
        
        # Relay
        elif (p := re.search(r'Prueba\s.+\.,\s*(4 x 50m)\s*(Libre|Estilos).*', line)):
            prueba = p.group(1) + p.group(2)
        
        # Look for the name of the swimmer
        elif (s := re.search(r'[0-9](.+,\s.+)\s\d{2}\s(.+)\s(.+)\s_.+$', line)):
            swimmer = s.group(1)
            club = s.group(2)
            time = s.group(3)
            data.append((swimmer, gender, club, prueba, time))

        # Look for the name of the relay
        elif (s := re.search(r'[0-9](.+\s.+)\s\d{2}\s(.+)\s(.+)\s_.+$', line)):
            swimmer = s.group(1)
            club = s.group(2)
            time = s.group(3)
            data.append((swimmer, gender, club, prueba, time))
data

[('GARCIA HERNANDEZ, Ines',
  'Femenino',
  'C. Alameda De Osuna',
  '50m Libre',
  '30.02'),
 ('POP MARTIN, Alexia', 'Femenino', 'C.M.N. Arganda', '50m Libre', '28.96'),
 ('BLANCO PEÑAS, Vega', 'Femenino', 'C.N. Jimenez', '50m Libre', '29.00'),
 ('VERDEJO RODRIGUEZ, Paula Haotong',
  'Femenino',
  'C.N. Vallmont',
  '50m Libre',
  'NT'),
 ('CONTRERAS CASTILLO, Sandra',
  'Femenino',
  'C.N. San Blas',
  '50m Libre',
  '28.60'),
 ('GARRIDO PEDROSA, Natalia',
  'Femenino',
  'A.D.N. Mostoles',
  '50m Libre',
  '27.95'),
 ('FLORES ALVAREZ, Lucia',
  'Femenino',
  'C.N. Ciudad De Alcorcon',
  '50m Libre',
  '27.36'),
 ('BELCIUG BELCIUG, Bianca Claudia',
  'Femenino',
  'C.N. Leganes',
  '50m Libre',
  '27.59'),
 ('SANCHEZ-PATON GOMEZ, Maria',
  'Femenino',
  'C. Alameda De Osuna',
  '50m Libre',
  '28.51'),
 ('CASTRO HIDALGO, Jimena',
  'Femenino',
  'C.M.N. Arganda',
  '50m Libre',
  '28.78'),
 ('BOLAÑOS MARTINEZ, Sara', 'Femenino', 'C.N. Leganes', '50m Libre', '27.20'),
 ('MAYORDOMO DEL

In [43]:
import pandas as pd

df = pd.DataFrame(data, columns= ['Name', 'Gender','Club', 'Prueba', 'Time'])
df

,Name,Gender,Club,Prueba,Time
0,"GARCIA HERNANDEZ, Ines",Femenino,C. Alameda De Osuna,50m Libre,30.02
1,"POP MARTIN, Alexia",Femenino,C.M.N. Arganda,50m Libre,28.96
2,"BLANCO PEÑAS, Vega",Femenino,C.N. Jimenez,50m Libre,29.00
3,"VERDEJO RODRIGUEZ, Paula Haotong",Femenino,C.N. Vallmont,50m Libre,NT
4,"CONTRERAS CASTILLO, Sandra",Femenino,C.N. San Blas,50m Libre,28.60
...,...,...,...,...,...
496,C.M.N. Arganda,Femenino,C.M.N. Arganda,4 x 50mEstilos,2:08.47
497,C.N. Leganes,Femenino,C.N. Leganes,4 x 50mEstilos,2:04.38
498,C.N. Ciudad De Alcorcon,Femenino,C.N. Ciudad De Alcorcon,4 x 50mEstilos,2:04.03
499,C.N. Jimenez,Femenino,C.N. Jimenez,4 x 50mEstilos,2:04.09


In [44]:
df['Time'].replace('NT', '59:59.59', inplace= True)

C:\Users\jesus\AppData\Local\Temp\ipykernel_2504\2302876258.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Time'].replace('NT', '59:59.59', inplace= True)


In [45]:
df['Time'] = df['Time'].apply(lambda x: int(x[0:x.index(':')]) * 60 + int(x[x.index(':')+1:x.index('.')]) + float('0.'+x[-2:]) if len(x) > 5 else float(x))

In [46]:
gender = 'Masculino'

punctuations = [22,17,14,13,12,11,10,9,8,7,6,5,4,3,2,1]

df = df[df['Gender'] == gender]

points = {club : 0 for club in df['Club'].unique()}

for prueba in df['Prueba'].unique():

    swimmers = df[(df['Prueba'] == prueba) & (df['Gender'] == gender)]

    swimmers.sort_values(by= 'Time', ascending= True, inplace= True)

    for i in range(swimmers.shape[0]):

        linea = swimmers.iloc[i]
        # Relay
        if 'x' in prueba:
            points[linea['Club']] += 2 * punctuations[i]
        else:
            points[linea['Club']] += punctuations[i]


points

C:\Users\jesus\AppData\Local\Temp\ipykernel_2504\3050937202.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swimmers.sort_values(by= 'Time', ascending= True, inplace= True)


{'C.N. Mirasierra': 353,
 'A.D. Rivas Natación': 289,
 'C.N. San Fernando De Henares': 258,
 'C.N. Coslada': 260,
 'C.N. Colegio Mirasur': 342,
 'C.N. San Blas': 303,
 'Safa Madrid': 396,
 'C.D. Sek': 462}

In [47]:
results = sorted(points.items(), key= lambda x: -x[1])
results

[('C.D. Sek', 462),
 ('Safa Madrid', 396),
 ('C.N. Mirasierra', 353),
 ('C.N. Colegio Mirasur', 342),
 ('C.N. San Blas', 303),
 ('A.D. Rivas Natación', 289),
 ('C.N. Coslada', 260),
 ('C.N. San Fernando De Henares', 258)]

In [42]:
df[df['Club']=='Safa Madrid']#['Name'].unique()

,Name,Gender,Club,Prueba,Time
26,"FERNANDEZ BLAZQUEZ, Victor",Masculino,Safa Madrid,50m Libre,23.74
27,"FERNANDEZ BLAZQUEZ, Pablo",Masculino,Safa Madrid,50m Libre,23.15
47,"ALDERETE GARCIA, Rodrigo",Masculino,Safa Madrid,1500m Libre,17:26.15
70,"MARTIN DE LA IGLESIA, David",Masculino,Safa Madrid,50m Braza,29.99
76,"GUTIERREZ MADERUELO, Javier",Masculino,Safa Madrid,50m Braza,29.40
100,"GARCIA NAVAS, Pablo",Masculino,Safa Madrid,100m Mariposa,59.34
109,"FERNANDEZ BLAZQUEZ, Pablo",Masculino,Safa Madrid,100m Mariposa,56.20
133,"FERNANDEZ BLAZQUEZ, Victor",Masculino,Safa Madrid,100m Espalda,59.09
136,"RODRIGUEZ CRESPO, Alejandro",Masculino,Safa Madrid,100m Espalda,1:00.53
164,"RUBIO DE PABLOS, Adrian",Masculino,Safa Madrid,200m Estilos,2:13.89
